In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import numpy as np
import pandas as pd

pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 6)
pd.set_option("display.width", None)

data_dir = f"../datasets/march-machine-learning-mania-2025"

In [2]:
raw = pd.DataFrame()

for fn in ["MNCAATourney", "MRegularSeason", "WNCAATourney", "WRegularSeason"]:
    raw = pd.concat(
        [
            raw,
            pd.read_csv(
                f"{data_dir}/{fn}DetailedResults.csv",
            ),
        ]
    ).reset_index(drop=True)

for c in raw.select_dtypes("int"):
    raw[c] = raw[c].astype("int32")

print(f"raw {raw.shape}")

raw (202033, 34)


In [3]:
tar = raw[["Season", "DayNum", "WTeamID", "LTeamID"]]
tar["WMargin"] = raw["WScore"] - raw["LScore"]
tar["LMargin"] = -tar["WMargin"]

tar = pd.concat(
    [
        tar.rename(
            columns={
                "WTeamID": "TeamID",
                "LTeamID": "OppID",
                "WMargin": "Margin",
            }
        ).drop(columns="LMargin"),
        tar.rename(
            columns={
                "LTeamID": "TeamID",
                "WTeamID": "OppID",
                "LMargin": "Margin",
            }
        ).drop(columns="WMargin"),
    ]
).reset_index(drop=True)

print(f"tar {tar.shape}\n")
print(tar)

tar (404066, 5)

        Season  DayNum  TeamID  OppID  Margin
0         2003     134    1421   1411       8
1         2003     136    1112   1436      29
2         2003     136    1113   1272      13
...        ...     ...     ...    ...     ...
404063    2025     120    3408   3412      -2
404064    2025     120    3187   3427     -19
404065    2025     120    3464   3460     -15

[404066 rows x 5 columns]


In [4]:
gam = raw.copy()
gam.loc[gam["WLoc"] == "A", "WLoc_"] = -1
gam.loc[gam["WLoc"] == "N", "WLoc_"] = 0
gam.loc[gam["WLoc"] == "H", "WLoc_"] = 1
gam = gam.drop(columns="WLoc").rename(columns={"WLoc_": "WLoc"})
gam["WLoc"] = gam["WLoc"].astype("int32")
gam["LLoc"] = -gam["WLoc"]

gam = pd.concat(
    [
        gam.rename(columns={c: f"{c[1:]}_gu" for c in gam if c[0] == "W"}).rename(
            columns={c: f"{c[1:]}_gt" for c in gam if c[0] == "L"}
        ),
        gam.rename(columns={c: f"{c[1:]}_gu" for c in gam if c[0] == "L"}).rename(
            columns={c: f"{c[1:]}_gt" for c in gam if c[0] == "W"}
        ),
    ]
).reset_index(drop=True)

gam = gam.rename(columns={"TeamID_gu": "TeamID", "TeamID_gt": "OppID"})
gam["NumOT_gu"] = gam["NumOT"]
gam = gam.drop(columns=["NumOT", "Loc_gt"])

print(f"gam {gam.shape}\n")
print(gam[[c for c in gam if c[-3:-1] != "_g"]])
print()
print(gam[[c for c in gam if c[-3:] == "_gu"]])
print()
print(gam[[c for c in gam if c[-3:] == "_gt"]])

gam (404066, 34)

        Season  DayNum  TeamID  OppID
0         2003     134    1421   1411
1         2003     136    1112   1436
2         2003     136    1113   1272
...        ...     ...     ...    ...
404063    2025     120    3408   3412
404064    2025     120    3187   3427
404065    2025     120    3464   3460

[404066 rows x 4 columns]

        Score_gu  FGM_gu  FGA_gu  FGM3_gu  FGA3_gu  FTM_gu  FTA_gu  OR_gu  DR_gu  Ast_gu  TO_gu  Stl_gu  Blk_gu  PF_gu  Loc_gu  NumOT_gu
0             92      32      69       11       29      17      26     14     30      17     12       5       3     22       0         1
1             80      31      66        7       23      11      14     11     36      22     16      10       7      8       0         0
2             84      31      59        6       14      16      22     10     27      18      9       7       4     19       0         0
...          ...     ...     ...      ...      ...     ...     ...    ...    ...     ...    ...     ..

In [5]:
sea = (
    gam.groupby(["Season", "TeamID"])
    .agg(
        dict(
            **{
                c: "sum"
                for c in gam.drop(
                    columns=[
                        "Season",
                        "TeamID",
                        "OppID",
                        "DayNum",
                    ]
                )
            },
            DayNum="count",
        )
    )
    .reset_index()
)

sea["DayNum"] = sea["DayNum"].astype("int32")

sea = sea.rename(columns={c: f"{c[:-2]}s{c[-1]}" for c in sea if c[-3:-1] == "_g"})

sea = sea.rename(columns={"DayNum": "Games_su"})

print(f"sea {sea.shape}\n")
print(sea[[c for c in sea if c[-3:-1] != "_s"]])
print()
print(sea[[c for c in sea if c[-3:] == "_su"]])
print()
print(sea[[c for c in sea if c[-3:] == "_st"]])

sea (13583, 33)

       Season  TeamID
0        2003    1102
1        2003    1103
2        2003    1104
...       ...     ...
13580    2025    3478
13581    2025    3479
13582    2025    3480

[13583 rows x 2 columns]

       Score_su  FGM_su  FGA_su  FGM3_su  FGA3_su  FTM_su  FTA_su  OR_su  DR_su  Ast_su  TO_su  Stl_su  Blk_su  PF_su  Loc_su  NumOT_su  Games_su
0          1603     536    1114      219      583     312     479    117    471     364    320     167      50    525       0         0        28
1          2127     733    1508      147      434     514     698    264    538     411    341     196      63    536       1         8        27
2          2002     695    1653      183      568     429     602    389    690     352    380     187     112    526       6         1        29
...         ...     ...     ...      ...      ...     ...     ...    ...    ...     ...    ...     ...     ...    ...     ...       ...       ...
13580      1519     506    1439      184      585 

In [6]:
gsx = pd.merge(gam, sea, on=["Season", "TeamID"])

gsx = gsx.rename(columns={c: f"T_{c}" for c in gsx if c[-3:-1] == "_s"})

gsx = pd.merge(
    gsx,
    sea,
    left_on=["Season", "OppID"],
    right_on=["Season", "TeamID"],
    suffixes=["", "_fromseason"],
)

gsx = gsx.drop(columns="TeamID_fromseason")

gsx = gsx.rename(
    columns={c: f"O_{c}" for c in gsx if c[:2] != "T_" and c[-3:-1] == "_s"}
)

print(f"gsx {gsx.shape}\n")
print(gsx[[c for c in gsx if c[-3:-1] not in ("_g", "_s")]])
print()
print(gsx[[c for c in gsx if c[-3:] == "_gu"]])
print()
print(gsx[[c for c in gsx if c[-3:] == "_gt"]])
print()
print(gsx[[c for c in gsx if c[:2] == "T_" and c[-3:] == "_su"]])
print()
print(gsx[[c for c in gsx if c[:2] == "T_" and c[-3:] == "_st"]])
print()
print(gsx[[c for c in gsx if c[:2] == "O_" and c[-3:] == "_su"]])
print()
print(gsx[[c for c in gsx if c[:2] == "O_" and c[-3:] == "_st"]])

gsx (404066, 96)

        Season  DayNum  TeamID  OppID
0         2003     134    1421   1411
1         2003     136    1112   1436
2         2003     136    1113   1272
...        ...     ...     ...    ...
404063    2025     120    3408   3412
404064    2025     120    3187   3427
404065    2025     120    3464   3460

[404066 rows x 4 columns]

        Score_gu  FGM_gu  FGA_gu  FGM3_gu  FGA3_gu  FTM_gu  FTA_gu  OR_gu  DR_gu  Ast_gu  TO_gu  Stl_gu  Blk_gu  PF_gu  Loc_gu  NumOT_gu
0             92      32      69       11       29      17      26     14     30      17     12       5       3     22       0         1
1             80      31      66        7       23      11      14     11     36      22     16      10       7      8       0         0
2             84      31      59        6       14      16      22     10     27      18      9       7       4     19       0         0
...          ...     ...     ...      ...      ...     ...     ...    ...    ...     ...    ...     ..

In [7]:
aop = (
    gsx.groupby(["Season", "TeamID"])[[c for c in gsx if c[:2] == "O_"]]
    .sum()
    .reset_index()
)

aop = aop.rename(columns={c: f"{c[2:-2]}a{c[-1]}" for c in aop if c[-3:-1] == "_s"})

print(f"aop {aop.shape}\n")
print(aop[[c for c in aop if c[-3:-1] != "_a"]])
print()
print(aop[[c for c in aop if c[-3:] == "_au"]])
print()
print(aop[[c for c in aop if c[-3:] == "_at"]])

aop (13583, 33)

       Season  TeamID
0        2003    1102
1        2003    1103
2        2003    1104
...       ...     ...
13580    2025    3478
13581    2025    3479
13582    2025    3480

[13583 rows x 2 columns]

       Score_au  FGM_au  FGA_au  FGM3_au  FGA3_au  FTM_au  FTA_au  OR_au  DR_au  Ast_au  TO_au  Stl_au  Blk_au  PF_au  Loc_au  NumOT_au  Games_au
0         57412   20042   44680     4964    14029   12364   17706   9378  18469   10938  11898    5643    2495  16224      61        31       807
1         54867   19010   42418     4843    13855   12004   17190   8669  17648   10362  11586    5235    2346  15703     -31        69       778
2         61173   21792   48500     5624    15822   11965   17130  10172  20372   12240  12597    6196    3160  15863     111        41       872
...         ...     ...     ...      ...      ...     ...     ...    ...    ...     ...    ...     ...     ...    ...     ...       ...       ...
13580     47238   17377   43155     4356    13993 

In [8]:
gsa = pd.merge(gsx, aop, on=["Season", "TeamID"])

stats = [c[:-3] for c in gam if c[-3:] == "_gu" and c not in ("Loc_gu", "NumOT_gu")]

for stat in stats:
    gsa[f"T_{stat}_su"] = gsa[f"T_{stat}_su"] - gsa[f"{stat}_gu"]
    gsa[f"T_{stat}_st"] = gsa[f"T_{stat}_st"] - gsa[f"{stat}_gt"]

    gsa[f"O_{stat}_su"] = gsa[f"O_{stat}_su"] - gsa[f"{stat}_gt"]
    gsa[f"O_{stat}_st"] = gsa[f"O_{stat}_st"] - gsa[f"{stat}_gu"]

    gsa[f"{stat}_au"] = gsa[f"{stat}_au"] - gsa[f"{stat}_gt"]
    gsa[f"{stat}_at"] = gsa[f"{stat}_at"] - gsa[f"{stat}_gu"]

for stat in ["Loc", "NumOT"]:
    gsa[f"T_{stat}_su"] = gsa[f"T_{stat}_su"] - gsa[f"{stat}_gu"]
    gsa[f"O_{stat}_su"] = gsa[f"O_{stat}_su"] + gsa[f"{stat}_gu"]
    gsa[f"{stat}_au"] = gsa[f"{stat}_au"] + gsa[f"{stat}_gu"]

gsa[f"T_Games_su"] -= 1
gsa[f"O_Games_su"] -= 1
gsa[f"Games_au"] -= 1

gsa = gsa.drop(columns=[c for c in gsa if c[-3:-1] == "_g"])

print(f"gsa {gsa.shape}\n")
print(gsa[[c for c in gsa if c[-3:-1] not in ("_s", "_a")]])
print()
print(gsa[[c for c in gsa if c[:2] == "T_" and c[-3:] == "_su"]])
print()
print(gsa[[c for c in gsa if c[:2] == "T_" and c[-3:] == "_st"]])
print()
print(gsa[[c for c in gsa if c[:2] == "O_" and c[-3:] == "_su"]])
print()
print(gsa[[c for c in gsa if c[:2] == "O_" and c[-3:] == "_st"]])
print()
print(gsa[[c for c in gsa if c[-3:] == "_au"]])
print()
print(gsa[[c for c in gsa if c[-3:] == "_at"]])

gsa (404066, 97)

        Season  DayNum  TeamID  OppID
0         2003     134    1421   1411
1         2003     136    1112   1436
2         2003     136    1113   1272
...        ...     ...     ...    ...
404063    2025     120    3408   3412
404064    2025     120    3187   3427
404065    2025     120    3464   3460

[404066 rows x 4 columns]

        T_Score_su  T_FGM_su  T_FGA_su  T_FGM3_su  T_FGA3_su  T_FTM_su  T_FTA_su  T_OR_su  T_DR_su  T_Ast_su  T_TO_su  T_Stl_su  T_Blk_su  T_PF_su  T_Loc_su  T_NumOT_su  T_Games_su
0             2126       730      1709        193        543       473       619      369      688       383      481       214        90      570        -4           5          30
1             2645       942      2045        221        622       540       763      468      859       547      452       264       129      551         3           4          31
2             2279       817      1719        120        380       525       783      414      690       46

In [9]:
pg = gsa.copy()

for stat in stats:
    pg[f"T_{stat}_su"] = (pg[f"T_{stat}_su"] / pg[f"T_Games_su"]).astype("float32")
    pg[f"T_{stat}_st"] = (pg[f"T_{stat}_st"] / pg[f"T_Games_su"]).astype("float32")

    pg[f"O_{stat}_su"] = (pg[f"O_{stat}_su"] / pg[f"O_Games_su"]).astype("float32")
    pg[f"O_{stat}_st"] = (pg[f"O_{stat}_st"] / pg[f"O_Games_su"]).astype("float32")

    pg[f"{stat}_au"] = (pg[f"{stat}_au"] / pg[f"Games_au"]).astype("float32")
    pg[f"{stat}_at"] = (pg[f"{stat}_at"] / pg[f"Games_au"]).astype("float32")

for stat in ["Loc", "NumOT"]:
    pg[f"T_{stat}_su"] = (pg[f"T_{stat}_su"] / pg[f"T_Games_su"]).astype("float32")
    pg[f"O_{stat}_su"] = (pg[f"O_{stat}_su"] / pg[f"O_Games_su"]).astype("float32")
    pg[f"{stat}_au"] = (pg[f"{stat}_au"] / pg[f"Games_au"]).astype("float32")

pg = pg.drop(columns=["T_Games_su", "O_Games_su", "Games_au"])

print(f"pg {pg.shape}\n")
print(pg[[c for c in pg if c[-3:-1] not in ("_s", "_a")]])
print()
print(pg[[c for c in pg if c[:2] == "T_" and c[-3:] == "_su"]])
print()
print(pg[[c for c in pg if c[:2] == "T_" and c[-3:] == "_st"]])
print()
print(pg[[c for c in pg if c[:2] == "O_" and c[-3:] == "_su"]])
print()
print(pg[[c for c in pg if c[:2] == "O_" and c[-3:] == "_st"]])
print()
print(pg[[c for c in pg if c[-3:] == "_au"]])
print()
print(pg[[c for c in pg if c[-3:] == "_at"]])

pg (404066, 94)

        Season  DayNum  TeamID  OppID
0         2003     134    1421   1411
1         2003     136    1112   1436
2         2003     136    1113   1272
...        ...     ...     ...    ...
404063    2025     120    3408   3412
404064    2025     120    3187   3427
404065    2025     120    3464   3460

[404066 rows x 4 columns]

        T_Score_su   T_FGM_su   T_FGA_su  T_FGM3_su  T_FGA3_su   T_FTM_su   T_FTA_su    T_OR_su    T_DR_su   T_Ast_su    T_TO_su  T_Stl_su  T_Blk_su    T_PF_su  T_Loc_su  T_NumOT_su
0        70.866669  24.333334  56.966667   6.433333  18.100000  15.766666  20.633333  12.300000  22.933332  12.766666  16.033333  7.133333  3.000000  19.000000 -0.133333    0.166667
1        85.322578  30.387096  65.967743   7.129032  20.064516  17.419355  24.612904  15.096774  27.709677  17.645161  14.580646  8.516129  4.161290  17.774193  0.096774    0.129032
2        75.966667  27.233334  57.299999   4.000000  12.666667  17.500000  26.100000  13.800000  23.00000

In [10]:
train = pd.merge(tar, pg, on=["Season", "DayNum", "TeamID"], suffixes=["", "_"])
train = train.drop(columns="OppID_")
train.to_csv("train_daynum.csv", index=False)
print(f"train {train.shape}")
print(train)

train (404066, 95)
        Season  DayNum  TeamID  OppID  Margin  T_Score_su  T_Score_st   T_FGM_su   T_FGA_su  T_FGM3_su  T_FGA3_su   T_FTM_su   T_FTA_su    T_OR_su    T_DR_su   T_Ast_su    T_TO_su  T_Stl_su  T_Blk_su    T_PF_su   T_FGM_st   T_FGA_st  T_FGM3_st  T_FGA3_st   T_FTM_st   T_FTA_st    T_OR_st    T_DR_st   T_Ast_st    T_TO_st   T_Stl_st  T_Blk_st    T_PF_st  T_Loc_su  T_NumOT_su  O_Score_su  O_Score_st   O_FGM_su   O_FGA_su  O_FGM3_su  O_FGA3_su   O_FTM_su   O_FTA_su    O_OR_su    O_DR_su   O_Ast_su    O_TO_su  O_Stl_su  O_Blk_su    O_PF_su   O_FGM_st   O_FGA_st  O_FGM3_st  O_FGA3_st   O_FTM_st   O_FTA_st    O_OR_st    O_DR_st   O_Ast_st    O_TO_st  O_Stl_st  O_Blk_st    O_PF_st  O_Loc_su  O_NumOT_su   Score_au   Score_at     FGM_au     FGA_au   FGM3_au    FGA3_au     FTM_au     FTA_au      OR_au      DR_au     Ast_au      TO_au    Stl_au    Blk_au      PF_au     FGM_at     FGA_at   FGM3_at    FGA3_at     FTM_at     FTA_at      OR_at      DR_at     Ast_at      TO_at    Stl_